## Setup PySpark on GPU & Install Libraries

In [1]:
!nvidia-smi

Thu Nov 23 15:42:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!rm spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

--2023-11-23 15:42:54--  https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228834641 (218M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop3.2.tgz’

spark-3.1.2-bin-had 100%[===================>] 218.23M  2.16MB/s    in 2m 5s   

2023-11-23 15:44:59 (1.75 MB/s) - ‘spark-3.1.2-bin-hadoop3.2.tgz’ saved [228834641/228834641]



In [3]:
!wget https://repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.12/21.12.0/rapids-4-spark_2.12-21.12.0.jar
!wget https://repo1.maven.org/maven2/ai/rapids/cudf/21.12.2/cudf-21.12.2-cuda11.jar

--2023-11-23 15:45:18--  https://repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.12/21.12.0/rapids-4-spark_2.12-21.12.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34365430 (33M) [application/java-archive]
Saving to: ‘rapids-4-spark_2.12-21.12.0.jar’

rapids-4-spark_2.12 100%[===================>]  32.77M   178MB/s    in 0.2s    

2023-11-23 15:45:19 (178 MB/s) - ‘rapids-4-spark_2.12-21.12.0.jar’ saved [34365430/34365430]

--2023-11-23 15:45:19--  https://repo1.maven.org/maven2/ai/rapids/cudf/21.12.2/cudf-21.12.2-cuda11.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 328667466 (313M) [appli

In [4]:
import os

os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['SPARK_HOME'] = "/kaggle/working/spark-3.1.2-bin-hadoop3.2"

os.environ['PYSPARK_SUBMIT_ARGS'] = "--jars /kaggle/working/rapids-4-spark_2.12-21.12.0.jar,/kaggle/working/cudf-21.12.2-cuda11.jar --master local[*] pyspark-shell"

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

import findspark
findspark.init()

import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col, array_contains, udf, when, to_timestamp, date_format, from_unixtime, min, max, unix_timestamp, lit, year, month
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Configurations

In [5]:
ss = SparkSession.builder.\
            appName("H&M-Recommendations").\
            config("spark.sql.files.maxPartitionBytes", 5000000).\
            config('spark.plugins','com.nvidia.spark.SQLPlugin').\
            getOrCreate()

ss.sparkContext.addPyFile('/kaggle/working/rapids-4-spark_2.12-21.12.0.jar')
ss.sparkContext.addPyFile('/kaggle/working/cudf-21.12.2-cuda11.jar')
ss.conf.set('spark.rapids.sql.enabled','true')
ss.conf.set('spark.rapids.sql.incompatibleOps.enabled', 'true')
ss.conf.set('spark.rapids.sql.format.csv.read.enabled', 'true')
ss.conf.set('spark.rapids.sql.format.csv.enabled', 'true')

spark = SparkSession(ss)

23/11/23 15:45:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/23 15:45:26 WARN RapidsPluginUtils: RAPIDS Accelerator 21.12.0 using cudf 21.12.2. To disable GPU support set `spark.rapids.sql.enabled` to false
23/11/23 15:45:35 WARN Plugin: Installing rapids UDF compiler extensions to Spark. The compiler is disabled by default. To enable it, set `spark.rapids.sql.udfCompiler.enabled` to true


In [6]:
spark

## Load Data

In [7]:
dir_path = '/kaggle/input/h-and-m-personalized-fashion-recommendations/'
trnsctn_data_path = dir_path + 'transactions_train.csv'

transaction = spark.read.\
                option("header",True).\
                csv(trnsctn_data_path)

transaction.printSchema()

root
 |-- t_dat: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- sales_channel_id: string (nullable = true)



## Data Preparation

Transaction period

In [8]:
min_date, max_date = transaction.select(min("t_dat"), max("t_dat")).first()
min_date, max_date

('2018-09-20', '2020-09-22')

Due to memory limitation, select only the 2020 data

In [9]:
cus_atcl_grp =  transaction.withColumn('t_dat', transaction['t_dat'].cast('string'))
cus_atcl_grp = cus_atcl_grp.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
cus_atcl_grp = cus_atcl_grp.withColumn('year', year(col('date')))
cus_atcl_grp = cus_atcl_grp.withColumn('month', month(col('date')))
cus_atcl_grp = cus_atcl_grp.withColumn('day', date_format(col('date'), "d"))

cus_atcl_grp = cus_atcl_grp[cus_atcl_grp['year'] == 2020]
cus_atcl_grp = cus_atcl_grp[cus_atcl_grp['month'] == 9]
cus_atcl_grp = cus_atcl_grp[cus_atcl_grp['day'] == 22]

# Remove all blocks for transaction data from memory and disk
transaction.unpersist()

cus_atcl_grp = cus_atcl_grp.groupby('customer_id', 'article_id').count()

cus_atcl_grp.show(5)

+--------------------+----------+-----+
|         customer_id|article_id|count|
+--------------------+----------+-----+
|850ec77661a417d27...|0942187001|    2|
|d1c5629d6d7e39067...|0751471042|    1|
|e4efdaca582acbae2...|0880478003|    1|
|2e909b47d57f46c92...|0840016002|    1|
|aac0b270413320504...|0784926001|    1|
+--------------------+----------+-----+
only showing top 5 rows



In [10]:
print(f"Shape of customer article group: {(cus_atcl_grp.count(), len(cus_atcl_grp.columns))}")

Shape of customer article group: (29486, 3)


Compute the sparsity of customer x article matrix

In [11]:
# Count the total number of article in the dataset
numerator = cus_atcl_grp.select("count").count()

# Count the number of distinct customerid and distinct articleid
num_users = cus_atcl_grp.select("customer_id").distinct().count()
num_articles = cus_atcl_grp.select("article_id").distinct().count()
print(f"No of distinct customers: {num_users}")
print(f"No of distinct articles: {num_articles}")

# Set the denominator equal to the number of customer multiplied by the number of articles
denominator = num_users * num_articles

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator * 1.0) / denominator) * 100
print(f"Sparsity: {sparsity:.2f}%")

No of distinct customers: 10528
No of distinct articles: 7971
Sparsity: 99.96%


Most frequent customers

In [12]:
userId_count = cus_atcl_grp.groupBy("customer_id").count().orderBy('count', ascending=False)
userId_count.show()

+--------------------+-----+
|         customer_id|count|
+--------------------+-----+
|30b6056bacc5f5c9d...|   28|
|5e8fb4d457fdffc61...|   28|
|dc1b173e541f8d3c1...|   27|
|6335d496ef463bc40...|   25|
|1796e87fd2e88932b...|   25|
|f50287d9cf052d4b4...|   24|
|54e8ebd39543b5a4d...|   23|
|fd5ce8716faf00f6a...|   23|
|850ec77661a417d27...|   22|
|fc783381f1ea2174c...|   21|
|a08e284bb18add2d7...|   21|
|b49647f84a99ced53...|   21|
|383e1b07e2c1fe169...|   21|
|298523b6637340717...|   21|
|ad3663a848dccbdda...|   21|
|32f3a6a7ce63d302c...|   21|
|b606fe5786c00151a...|   21|
|3ca77aab50ae4532b...|   20|
|2a721767cd9864ed5...|   20|
|9827b2c9a35696193...|   19|
+--------------------+-----+
only showing top 20 rows



Most purchased articles

In [13]:
articleId_count = cus_atcl_grp.groupBy("article_id").count().orderBy('count', ascending=False)
articleId_count.show()

+----------+-----+
|article_id|count|
+----------+-----+
|0924243002|   91|
|0918522001|   88|
|0866731001|   78|
|0751471001|   75|
|0448509014|   73|
|0714790020|   72|
|0762846027|   68|
|0928206001|   67|
|0893432002|   66|
|0918292001|   65|
|0915529005|   64|
|0788575004|   63|
|0915529003|   63|
|0863583001|   60|
|0930380001|   59|
|0573085028|   59|
|0919273002|   58|
|0850917001|   57|
|0573085042|   56|
|0874110016|   53|
+----------+-----+
only showing top 20 rows



## Data Pre-processing

Convert customer_id and article_id columns in index form

In [14]:
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(cus_atcl_grp.columns) - set(['count']))]
pipeline = Pipeline(stages=indexer)
cus_atcl_grp_trnsfmd = pipeline.fit(cus_atcl_grp).transform(cus_atcl_grp)
cus_atcl_grp_trnsfmd.show()

23/11/23 15:53:18 WARN DAGScheduler: Broadcasting large task binary with size 1209.2 KiB


+--------------------+----------+-----+-----------------+----------------+
|         customer_id|article_id|count|customer_id_index|article_id_index|
+--------------------+----------+-----+-----------------+----------------+
|0333e5dda0257e9f4...|0839332002|    2|           4098.0|           732.0|
|298b9047df341b83f...|0928719001|    1|           7298.0|          2683.0|
|276f909c679d4ae2a...|0540701003|    1|            151.0|          2091.0|
|166546028742fe655...|0767423013|    1|            814.0|           974.0|
|081373184e601470c...|0714790020|    1|           4146.0|             5.0|
|0be4f1ecce204ee32...|0573085028|    1|            799.0|            14.0|
|0fc371e67e61a31d7...|0907170001|    1|           1737.0|          1978.0|
|09bec2a61046ccbea...|0860336002|    1|           6792.0|          2368.0|
|1918933afff08e955...|0914672001|    1|            285.0|          7685.0|
|14a298482fa7f9d52...|0894353002|    1|             87.0|          3772.0|
|21de2945b01506779...|082

## Create Training and Test Data

In [15]:
(training, test) = cus_atcl_grp_trnsfmd.randomSplit([0.7, 0.3])

## ALS Model Training

PySpark ALS: [Documentation](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.recommendation.ALS.html)

Following below steps to build the model
* Set nonnegative as ‘True’, since we are looking count greater than 0.
* Since we are working with explicit, set it to implicit ratings to ‘False’ or by default it takes explicit.
* Set "coldStartStrategy" to "drop" to ensure customers and/or articles in the evaluation set, that are not in training set are dropped.

Most important hyper-params in Alternating Least Square (ALS):

* **maxIter**: the maximum number of iterations to run (defaults to 10)
* **rank**: the number of latent factors in the model (defaults to 10)
* **regParam**: the regularization parameter in ALS (defaults to 1.0)

In [16]:
# Create ALS model
als = ALS(userCol="customer_id_index", itemCol="article_id_index", ratingCol="count",
          coldStartStrategy="drop", nonnegative=True)

# Tune model using ParamGridBuilder
param_grid = ParamGridBuilder()\
            .addGrid(als.rank, [10])\
            .addGrid(als.maxIter,[5, 10])\
            .addGrid(als.regParam,[0.01])\
            .build()

# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName = "rmse",labelCol = 'count', predictionCol = 'prediction')

# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator,numFolds=3)


# Fit ALS model to training data
model = cv.fit(training)

23/11/23 15:53:59 WARN DAGScheduler: Broadcasting large task binary with size 1239.2 KiB
23/11/23 15:54:00 WARN DAGScheduler: Broadcasting large task binary with size 1240.7 KiB
23/11/23 15:54:04 WARN DAGScheduler: Broadcasting large task binary with size 1242.2 KiB
23/11/23 15:54:07 WARN DAGScheduler: Broadcasting large task binary with size 1243.5 KiB
23/11/23 15:54:07 WARN DAGScheduler: Broadcasting large task binary with size 1242.5 KiB
23/11/23 15:54:08 WARN DAGScheduler: Broadcasting large task binary with size 1243.8 KiB
23/11/23 15:54:09 WARN DAGScheduler: Broadcasting large task binary with size 1244.5 KiB
23/11/23 15:54:09 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/11/23 15:54:09 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/11/23 15:54:09 WARN DAGScheduler: Broadcasting large task binary with size 1247.6 KiB
23/11/23 15:54:09 WARN DAGScheduler: Broadcasting large task binary with size

## Model Evaluation

In [17]:
best_model = model.bestModel

# Generate predictions and evaluate using RMSE
predictions = best_model.transform(test)
rmse = evaluator.evaluate(predictions)

23/11/23 16:00:01 WARN DAGScheduler: Broadcasting large task binary with size 1273.2 KiB
23/11/23 16:00:01 WARN DAGScheduler: Broadcasting large task binary with size 1274.6 KiB
23/11/23 16:00:42 WARN DAGScheduler: Broadcasting large task binary with size 1221.9 KiB
23/11/23 16:00:45 WARN DAGScheduler: Broadcasting large task binary with size 1305.3 KiB
23/11/23 16:00:48 WARN DAGScheduler: Broadcasting large task binary with size 1335.0 KiB
23/11/23 16:00:50 WARN DAGScheduler: Broadcasting large task binary with size 1335.9 KiB


In [19]:
print(f"RMSE = {rmse:.4f}")
print("**Best Model**")
print("Rank : {}".format(best_model.rank))
print("MaxIter: {}".format(best_model._java_obj.parent().getMaxIter()))
print("RegParam: {}".format(best_model._java_obj.parent().getRegParam()))

RMSE = 0.5738
**Best Model**
Rank : 10
MaxIter: 10
RegParam: 0.01


## Inferencing

#### Recommendations by article ID

In [20]:
user_recs = best_model.recommendForAllItems(10).show(10)

23/11/23 16:01:26 WARN DAGScheduler: Broadcasting large task binary with size 1329.7 KiB


+----------------+--------------------+
|article_id_index|     recommendations|
+----------------+--------------------+
|            1580|[{4907, 5.7840943...|
|            4900|[{3925, 2.4282556...|
|            5300|[{5127, 2.4253845...|
|            6620|[{4979, 4.4764447...|
|            7240|[{4979, 2.9939222...|
|            7340|[{4907, 5.57905},...|
|             471|[{4907, 7.6620617...|
|            1591|[{4907, 6.0614357...|
|            4101|[{4907, 6.7523165...|
|            1342|[{4907, 5.018008}...|
+----------------+--------------------+
only showing top 10 rows



23/11/23 16:01:33 WARN DAGScheduler: Broadcasting large task binary with size 1298.5 KiB


#### Recommendations by customer ID

In [21]:
df_recom = best_model.recommendForAllUsers(10)
df_recom.show(10)

23/11/23 16:01:34 WARN DAGScheduler: Broadcasting large task binary with size 1329.7 KiB


+-----------------+--------------------+
|customer_id_index|     recommendations|
+-----------------+--------------------+
|             1580|[{4146, 3.8156264...|
|             4900|[{4780, 2.5283751...|
|             5300|[{4146, 3.1368093...|
|             6620|[{368, 3.4642432}...|
|             7240|[{1203, 3.1038446...|
|             7340|[{2511, 3.1514034...|
|              471|[{3870, 4.207459}...|
|             1591|[{368, 3.5396628}...|
|             4101|[{5162, 1.6550583...|
|             1342|[{4146, 3.7450473...|
+-----------------+--------------------+
only showing top 10 rows



23/11/23 16:01:40 WARN DAGScheduler: Broadcasting large task binary with size 1298.5 KiB


In [22]:
df_recom = df_recom.select("customer_id_index","recommendations.article_id_index")
df_recom.show(10)
df_recom = df_recom.toPandas()

23/11/23 16:01:40 WARN DAGScheduler: Broadcasting large task binary with size 1329.7 KiB
23/11/23 16:01:47 WARN DAGScheduler: Broadcasting large task binary with size 1298.7 KiB


+-----------------+--------------------+
|customer_id_index|    article_id_index|
+-----------------+--------------------+
|             1580|[4146, 6383, 5040...|
|             4900|[4780, 2511, 3013...|
|             5300|[4146, 3870, 2511...|
|             6620|[368, 7231, 6849,...|
|             7240|[1203, 4780, 734,...|
|             7340|[2511, 6383, 5040...|
|              471|[3870, 3869, 7603...|
|             1591|[368, 7231, 6383,...|
|             4101|[5162, 368, 2319,...|
|             1342|[4146, 368, 3870,...|
+-----------------+--------------------+
only showing top 10 rows



23/11/23 16:01:47 WARN DAGScheduler: Broadcasting large task binary with size 1329.7 KiB
23/11/23 16:01:53 WARN DAGScheduler: Broadcasting large task binary with size 1298.8 KiB


In [23]:
df_recom.sort_values('customer_id_index')

,customer_id_index,article_id_index
8723,0,"[3870, 4405, 3869, 4780, 1198, 7093, 7092, 251..."
2009,1,"[5040, 6383, 368, 2511, 610, 4146, 7828, 6414,..."
7990,2,"[4146, 3870, 2511, 1564, 3869, 4405, 1198, 489..."
2389,3,"[722, 3858, 2225, 368, 3870, 6950, 1341, 6702,..."
4721,4,"[368, 1564, 2978, 3870, 4146, 1151, 5162, 1920..."
...,...,...
3114,10520,"[368, 2511, 5040, 6383, 3870, 722, 4146, 4780,..."
7528,10521,"[2163, 7828, 6414, 1198, 6533, 3237, 2571, 638..."
5555,10522,"[3870, 2511, 4780, 3869, 765, 1198, 996, 1910,..."
5746,10525,"[4146, 2511, 1765, 598, 3870, 2149, 7603, 3764..."


Convert string to index

In [24]:
md = cus_atcl_grp_trnsfmd.select(cus_atcl_grp_trnsfmd['article_id'],
                                 cus_atcl_grp_trnsfmd['article_id_index'],
                                 cus_atcl_grp_trnsfmd['customer_id'],
                                 cus_atcl_grp_trnsfmd['customer_id_index'])
md = md.toPandas()
md

23/11/23 16:02:53 WARN DAGScheduler: Broadcasting large task binary with size 1208.6 KiB


,article_id,article_id_index,customer_id,customer_id_index
0,0839332002,732.0,0333e5dda0257e9f498be52f1e569bfae576caed0cbdcd...,4098.0
1,0767423013,974.0,166546028742fe6558df416a834ef1ddf40582273f64e1...,814.0
2,0928719001,2683.0,298b9047df341b83fedd27e412428cf13ea1dd8cb9f7db...,7298.0
3,0907170001,1978.0,0fc371e67e61a31d71dd74aeec841e3ba386c168ed8d60...,1737.0
4,0573085028,14.0,0be4f1ecce204ee32feffd81e49f9d5e7e9eecd6621670...,799.0
...,...,...,...,...
29481,0850917001,17.0,93474da645072c1a4239bd3489f73e2e60838268dd423b...,2229.0
29482,0918892001,225.0,bb8972244c60820ce6d7517daf4d78b5162cf561a8b3f8...,9498.0
29483,0870548001,2402.0,6043cf59d8aa03c2cfa27ef2c2f59d42b4e7b54d806c58...,434.0
29484,0799365028,124.0,5b92f1686575669ef7510dcdd3fc3c9e6ae3c5a4068994...,2027.0


In [25]:
dict1 = dict(zip(md['article_id_index'],md['article_id']))
dict2 = dict(zip(md['customer_id_index'],md['customer_id']))
df_recom['article_id'] = df_recom['article_id_index'].map(lambda x: [dict1[y] for y in x if y in dict1])
df_recom['customer_id'] = df_recom['customer_id_index'].map(dict2)
df_recom

,customer_id_index,article_id_index,article_id,customer_id
0,1580,"[4146, 6383, 5040, 3870, 2511, 368, 6414, 3869...","[0316441001, 0857347002, 0750481010, 090496100...",db1f0328dcdc4e12318b7091159ea412af2934047fc191...
1,4900,"[4780, 2511, 3013, 3192, 368, 5040, 6383, 3870...","[0702932001, 0894481001, 0740213003, 080080300...",52354f66ed3ca4a7fa91fe0ac8ff38f82ed766327f155e...
2,5300,"[4146, 3870, 2511, 368, 3869, 7603, 610, 4780,...","[0316441001, 0904961003, 0894481001, 071749007...",79ade819f779b09f30e210d017a32023e5c33d46039e42...
3,6620,"[368, 7231, 6849, 7603, 2511, 2573, 610, 3870,...","[0717490070, 0892280002, 0876196007, 091025800...",fbab7712ccfd4fb67f7a8ffa7543eb43094df970b1f76b...
4,7240,"[1203, 4780, 734, 3192, 3013, 1741, 2978, 40, ...","[0863477001, 0702932001, 0842028004, 080080300...",25fd7087fa563e76c6f1b6988d23d3b03e105f7b3b9fc3...
...,...,...,...,...
9108,6078,"[6950, 2319, 3858, 6064, 722, 1203, 5891, 2716...","[0880059001, 0831684001, 0904076001, 083654000...",c4a3d6df75739a5368a7d3a34059e4c62641336bf4b915...
9109,6438,"[2511, 7231, 1564, 4146, 2149, 598, 3870, 3418...","[0894481001, 0892280002, 0902288001, 031644100...",e9832099d0a3bc5eb3a92b9e7ed2f61c979002571a1823...
9110,6638,"[3870, 368, 7603, 4780, 40, 2511, 3869, 2978, ...","[0904961003, 0717490070, 0910258001, 070293200...",fd472eb741eb69d3261df85dfa083a899c3bcd18da682b...
9111,89,"[4146, 2511, 4894, 1203, 368, 3870, 6950, 6064...","[0316441001, 0894481001, 0721990004, 086347700...",17f1c7de37eaee65e9d831fe36b7e32fce928d38880e7b...


In [26]:
recom_final = df_recom.drop(['customer_id_index','article_id_index'], axis = 1)
final_pred = recom_final[['customer_id','article_id']]
final_pred

,customer_id,article_id
0,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,"[0316441001, 0857347002, 0750481010, 090496100..."
1,52354f66ed3ca4a7fa91fe0ac8ff38f82ed766327f155e...,"[0702932001, 0894481001, 0740213003, 080080300..."
2,79ade819f779b09f30e210d017a32023e5c33d46039e42...,"[0316441001, 0904961003, 0894481001, 071749007..."
3,fbab7712ccfd4fb67f7a8ffa7543eb43094df970b1f76b...,"[0717490070, 0892280002, 0876196007, 091025800..."
4,25fd7087fa563e76c6f1b6988d23d3b03e105f7b3b9fc3...,"[0863477001, 0702932001, 0842028004, 080080300..."
...,...,...
9108,c4a3d6df75739a5368a7d3a34059e4c62641336bf4b915...,"[0880059001, 0831684001, 0904076001, 083654000..."
9109,e9832099d0a3bc5eb3a92b9e7ed2f61c979002571a1823...,"[0894481001, 0892280002, 0902288001, 031644100..."
9110,fd472eb741eb69d3261df85dfa083a899c3bcd18da682b...,"[0904961003, 0717490070, 0910258001, 070293200..."
9111,17f1c7de37eaee65e9d831fe36b7e32fce928d38880e7b...,"[0316441001, 0894481001, 0721990004, 086347700..."


## Submission

In [27]:
final_pred.to_csv('submission.csv', index=False)